# IndicTrans

In [1]:
import random

In [2]:
%cd /kaggle/working/
!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd /kaggle/working/IndicTransTokenizer
!pip install --editable ./

/kaggle/working
Cloning into 'IndicTransTokenizer'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 123 (delta 52), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (123/123), 3.85 MiB | 16.31 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/kaggle/working/IndicTransTokenizer
Obtaining file:///kaggle/working/IndicTransTokenizer
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-m4lw8qgz/indic-nlp-library-it2_7b013b4945ed48afb9c2d71e8f185d6c
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-m4lw8qgz/indic-nlp-library-it2_7b013b4945ed48afb9c2d71e8f185d6c
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 09d30a15286cc252a12682e5450c807379717eaf
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Import Statements
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

In [4]:
ls

IndicTransTokenizer/           LICENSE    requirements.txt
IndicTransTokenizer.egg-info/  README.md  setup.py


In [5]:
# Read Data from file
# Open the first file
with open('/kaggle/input/file-dataset/test.en', 'r') as f:
    en_dataset = f.readlines()

# Open the second file
with open('/kaggle/input/file-dataset/test.gu', 'r') as guj_dataset_2:
    guj_dataset = guj_dataset_2.readlines()

# Open the third file
with open('/kaggle/input/file-dataset/test.hi', 'r') as hin_dataset_3:
    hin_dataset = hin_dataset_3.readlines()


# Load the 1000 random sentence

In [5]:
# Load These sentences
def load_sentences(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            sentences.append(line.strip())  # Remove leading/trailing whitespace
    return sentences

# Load English sentences
en_sentences = load_sentences('/kaggle/input/dataset-2/distilled_english_sentences.txt')

# Load Hindi sentences
hi_sentences = load_sentences('/kaggle/input/dataset-2/distilled_hindi_sentences.txt')

# Load Gujarati sentences
gu_sentences = load_sentences('/kaggle/input/dataset-2/distilled_gujarati_sentences.txt')

# Print a sentence from each language as an example
print("English:", en_sentences[1])
print("Hindi:", hi_sentences[1])
print("Gujarati:", gu_sentences[1])

English: India has a robust strategic partnership with all the three countries.
Hindi: भारत के इन सभी तीन देशों के साथ मजबूत रणनीतिक रिश्ते रहे हैं.’
Gujarati: આ તમામ ત્રણેય દેશો સાથે ભારતની મજબુત વ્યુહાત્મક ભાગીદારી છે.


In [6]:
# Translate Function which will convert src to target
BATCH_SIZE = 4
def translate(model,ip,tokenizer,src,target,input_sentences):
    translation = []
    for i in range(0,len(input_sentences),BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]
        
        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src, tgt_lang=target)
        
        # Tokenize the batch and generate input encodings
        batch = tokenizer(batch, src=True, return_tensors="pt").to(device)

        with torch.inference_mode():
            outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

        # Decode the generated tokens into text
        outputs = tokenizer.batch_decode(outputs, src=False)
    
        # Postprocess the translations, including entity replacement
        outputs = ip.postprocess_batch(outputs, lang=target)
        translation.append(outputs)
        del batch
        torch.cuda.empty_cache()
    return translation

In [7]:
eng_code = 'eng_Latn'
hin_code =  'hin_Deva'
guj_code = 'guj_Gujr'

# English to Hindi

In [29]:
# # Model and Tokenizer for en-indic
tokenizer = IndicTransTokenizer(direction="en-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True,low_cpu_mem_usage=True)

device=torch.device('cuda',0)
model.to(device)

IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(32322, 512, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_

In [34]:
indictrans_en_hi = translate(model,ip,tokenizer,eng_code,hin_code,en_sentences)

In [55]:
# Flatten list 
indictrans_en_hi_corrected = [sentence for sublist in indictrans_en_hi for sentence in sublist]
print(indictrans_en_hi_corrected[1])

कर्नाटक में, विशेष रूप से दूर-दराज के स्थानों में, हजारों महिलाओं ने उनकी सेवाओं का लाभ उठाया।


In [64]:
import zipfile
from IPython.display import FileLink
import os
# Save output to a file
output_file = "indictrans_translated_en_hi.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(indictrans_en_hi_corrected))
# 

# Zip the file into the Kaggle output directory
zip_file = os.path.join(kaggle_output_dir, "indictrans_translated_en_hi.zip")
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(output_file)

# Provide a download link to the zip file
FileLink(zip_file)

/kaggle/working/indictrans_translated_en_hi.zip

In [60]:
!ls

IndicTransTokenizer	      indictrans_translated_en_hi.txt
IndicTransTokenizer.egg-info  requirements.txt
LICENSE			      setup.py
README.md		      translated_en_hi.txt


# Hindi to English

In [8]:
tokenizer = IndicTransTokenizer(direction="indic-en")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True,low_cpu_mem_usage=True)

device=torch.device('cuda',0)
model.to(device)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/914M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(122706, 512, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [9]:
indictrans_hi_en = translate(model,ip,tokenizer,hin_code,eng_code,hi_sentences)

In [12]:
# Flatten list 
indictrans_hi_en_corrected = [sentence for sublist in indictrans_hi_en for sentence in sublist]
print(len(indictrans_hi_en_corrected))

1000


In [15]:
import shutil
from IPython.display import FileLink
import os
# Save output to a file
output_file = "indictrans_translated_hi_en.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(indictrans_hi_en_corrected))

# Specify the Kaggle output directory
kaggle_output_dir = "/kaggle/working/"

# Move the file to the Kaggle output directory
shutil.move(output_file, os.path.join(kaggle_output_dir, output_file))

# Provide a download link to the file
FileLink(os.path.join(kaggle_output_dir, output_file))

/kaggle/working/indictrans_translated_hi_en.txt

# Gujrati to Hindi

In [17]:
tokenizer = IndicTransTokenizer(direction="indic-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True,low_cpu_mem_usage=True)

device=torch.device('cuda',0)
model.to(device)

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(122706, 512, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [18]:
indictrans_gu_hi = translate(model,ip,tokenizer,guj_code,hin_code,gu_sentences)

In [19]:
# Flatten list 
indictrans_gu_hi_corrected = [sentence for sublist in indictrans_gu_hi for sentence in sublist]
print(indictrans_gu_hi_corrected[1])

इन तीनों देशों के साथ भारत की मजबूत रणनीतिक साझेदारी है।


In [20]:
import zipfile
from IPython.display import FileLink
import os
# Save output to a file
output_file = "indictrans_translated_gu_hi.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(indictrans_gu_hi_corrected))
# 

# Zip the file into the Kaggle output directory
zip_file = os.path.join(kaggle_output_dir, "indictrans_translated_gu_hi.zip")
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(output_file)

# Provide a download link to the zip file
FileLink(zip_file)

/kaggle/working/indictrans_translated_gu_hi.zip

# Hindi to Gujrati

In [21]:
tokenizer = IndicTransTokenizer(direction="indic-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True,low_cpu_mem_usage=True)

device=torch.device('cuda',0)
model.to(device)

IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(122706, 512, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [22]:
indictrans_hi_gu = translate(model,ip,tokenizer,hin_code,guj_code,hi_sentences)

In [23]:
# Flatten list 
indictrans_hi_gu_corrected = [sentence for sublist in indictrans_hi_gu for sentence in sublist]
print(indictrans_hi_gu_corrected[1])

આ ત્રણેય દેશો સાથે ભારતના મજબૂત વ્યૂહાત્મક સંબંધો છે.


In [24]:
import zipfile
from IPython.display import FileLink
import os
# Save output to a file
output_file = "indictrans_translated_hi_gu.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(indictrans_hi_gu_corrected))
# 

# Zip the file into the Kaggle output directory
zip_file = os.path.join(kaggle_output_dir, "indictrans_translated_hi_gu.zip")
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(output_file)

# Provide a download link to the zip file
FileLink(zip_file)

/kaggle/working/indictrans_translated_hi_gu.zip